In [1]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")

Libraries imported successfully......


In [2]:
teams = pd.read_csv('expanded_teams_data.csv')

print("Data loaded successfully..")

Data loaded successfully..


In [3]:
teams

,code,team_id,team_name,team_short_name,unavailable,pulse_id,event_id,fixture_id,fixture_difficulty,opponent_team,opponent_score,team_score,Game_played,is_home
0,3,1,Arsenal,ARS,False,1,1,2.0,2.0,16.0,1.0,2.0,True,True
1,7,2,Aston Villa,AVL,False,2,1,6.0,4.0,15.0,5.0,1.0,True,False
2,91,3,Bournemouth,BOU,False,127,1,3.0,2.0,19.0,1.0,1.0,True,True
3,94,4,Brentford,BRE,False,130,1,8.0,3.0,18.0,2.0,2.0,True,True
4,36,5,Brighton,BHA,False,131,1,4.0,2.0,12.0,1.0,4.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,17,16,Nott'm Forest,NFO,False,15,38,374.0,2.0,6.0,NaN,NaN,False,False
756,49,17,Sheffield Utd,SHU,False,18,38,380.0,3.0,18.0,NaN,NaN,False,True
757,6,18,Spurs,TOT,False,21,38,380.0,2.0,17.0,NaN,NaN,False,False
758,21,19,West Ham,WHU,False,25,38,379.0,5.0,13.0,NaN,NaN,False,False


Games played


In [4]:
# Convert 'Game_played' column to integers (1 for True, 0 for False)
teams['Game_played'].fillna(False, inplace=True)
teams['Game_played'] = teams['Game_played'].astype(int)

# Calculate cumulative 'Games_Count' for each row
teams['Games_Count'] = teams.groupby(['team_id', 'team_name'])['Game_played'].cumsum()



Total goals

In [5]:
teams['Total_Goals'] = teams.groupby(['team_id', 'team_name'])['team_score'].cumsum()


Goals per game

In [6]:
teams['Goals_per_game'] = teams['Total_Goals'] / teams['Games_Count']

Goals conceded

In [7]:
teams['Total_Conceded'] = teams.groupby(['team_id', 'team_name'])['opponent_score'].cumsum()


In [8]:
teams['Conceded_per_game'] = teams['Total_Conceded'] / teams['Games_Count']

Win

In [9]:
teams['Win'] = teams['team_score'] > teams['opponent_score'] 
teams['Win'] = teams['Win'].astype(int)


Draw

In [10]:
teams['Draw'] = teams['team_score'] == teams['opponent_score'] 
teams['Draw'] = teams['Draw'].astype(int)

Loss

In [11]:
teams['Loss'] = teams['team_score'] < teams['opponent_score'] 
teams['Loss'] = teams['Loss'].astype(int)

Total wins

In [12]:
teams['Total_wins'] = teams.groupby(['team_id', 'team_name'])['Win'].cumsum()


Win percentage

In [13]:
teams['Win_percentage'] = (teams['Total_wins'] / teams['Games_Count']) * 100

Game Results

In [14]:
teams['result'] = teams.apply(lambda row: 1 if row['Win'] else (-1 if row['Loss'] else 0), axis=1)


Form

In [27]:
def calculate_form(group, num_games):
    forms = []
    current_form = ""
    
    for i, row in group.iterrows():
        result = row['result']
        if row['Game_played'] == True:
            if result == 1:
                current_form += 'W'
            elif result == -1:
                current_form += 'L'
            else:
                current_form += 'D'
            
            if len(current_form) > num_games:
                current_form = current_form[1:]
            
            forms.append(current_form)

        else:
            forms.append(None)
    return forms

# Calculate the form for each specific 'team_id' and assign it to the 'Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_forms = calculate_form(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Form'] = team_forms




Numeric Form

In [30]:
def calculate_numerical_form(group, num_games):
    numerical_forms = []
    current_form = []
    
    for i, row in group.iterrows():
        result = row['result']
        game_played = row['Game_played']
        
        if game_played:
            # Assign weights based on the position in the form
            weight = 1 + i / num_games
            
            if result == 1:
                value = 1
            elif result == -1:
                value = -1
            else:
                value = 0
            
            current_form.append(value * weight)
            
            if len(current_form) > num_games:
                current_form.pop(0)
            
            numerical_form = sum(current_form) / len(current_form)
            numerical_forms.append(numerical_form)
        
        else:
            numerical_forms.append(None)  # Append None for games that haven't been played
    
    return numerical_forms

# Calculate the numerical form for each specific 'team_id' and assign it to the 'Numerical_Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_numerical_forms = calculate_numerical_form(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Numerical_Form'] = team_numerical_forms

print(teams)


     code  team_id      team_name team_short_name  unavailable  pulse_id  \
0       3        1        Arsenal             ARS        False         1   
1       7        2    Aston Villa             AVL        False         2   
2      91        3    Bournemouth             BOU        False       127   
3      94        4      Brentford             BRE        False       130   
4      36        5       Brighton             BHA        False       131   
..    ...      ...            ...             ...          ...       ...   
755    17       16  Nott'm Forest             NFO        False        15   
756    49       17  Sheffield Utd             SHU        False        18   
757     6       18          Spurs             TOT        False        21   
758    21       19       West Ham             WHU        False        25   
759    39       20         Wolves             WOL        False        38   

     event_id  fixture_id  fixture_difficulty  opponent_team  ...  \
0           1     

In [31]:
Liverpool = teams[teams['team_id'] == 11]
Liverpool

,code,team_id,team_name,team_short_name,unavailable,pulse_id,event_id,fixture_id,fixture_difficulty,opponent_team,...,Total_Conceded,Conceded_per_game,Win,Draw,Loss,Total_wins,Win_percentage,result,Form,Numerical_Form
10,14,11,Liverpool,LIV,False,10,1,9.0,3.0,7.0,...,1.0,1.000000,0,1,0,0,0.000000,0,D,0.0
30,14,11,Liverpool,LIV,False,10,2,14.0,2.0,3.0,...,2.0,1.000000,1,0,0,1,50.000000,1,DW,3.5
50,14,11,Liverpool,LIV,False,10,3,29.0,4.0,15.0,...,3.0,1.000000,1,0,0,2,66.666667,1,DWW,6.0
70,14,11,Liverpool,LIV,False,10,4,37.0,3.0,2.0,...,3.0,0.750000,1,0,0,3,75.000000,1,DWWW,8.25
90,14,11,Liverpool,LIV,False,10,5,50.0,2.0,20.0,...,4.0,0.800000,1,0,0,4,80.000000,1,DWWWW,10.4
110,14,11,Liverpool,LIV,False,10,6,57.0,2.0,19.0,...,5.0,0.833333,1,0,0,5,83.333333,1,WWWWW,15.0
130,14,11,Liverpool,LIV,False,10,7,68.0,3.0,18.0,...,7.0,1.000000,0,0,1,5,71.428571,-1,WWWWL,8.2
150,14,11,Liverpool,LIV,False,10,8,72.0,3.0,5.0,...,NaN,NaN,0,0,0,5,71.428571,0,None,None
170,14,11,Liverpool,LIV,False,10,9,85.0,2.0,9.0,...,NaN,NaN,0,0,0,5,71.428571,0,None,None
190,14,11,Liverpool,LIV,False,10,10,97.0,2.0,16.0,...,NaN,NaN,0,0,0,5,71.428571,0,None,None
